# ML Tech Interview

Hello and welcome to the Machine Learning Tech Interview. This interview will be divided in two parts: the theoretical part and the practical/coding part. 

### **I will review only the scripts that will be sent (by pull request on this repo) by 1:00 pm**

Good Luck!!

## Theoretical Part

Please answer the following questions. 

#### What are the assumptions of a linear model (or any other type of model)?

The linear model assumes there is a linear relashionship between the features and the target.
For the model to work well there must not be significant collinearity between features, no auto-correlation, multivariate normality in the distribution of residuals and the variance around the regression line is the same for the predicted values. 

#### What’s the difference between K Nearest Neighbor and K-means Clustering?

The KNN gets the nearest points in the sample (classification supervised learning), while the K-means clusters according to the mean distance of a certain group (clustering unsupervised learning). 

#### How do you address overfitting?

By getting a bigger dataset (maybe by augmenting, but it is not always the best choice); trying a different model that generalizes better, reducing complexity; do some feature regularization. 

#### Explain Naive Bayes algorithms.

It is a supervised learning classification method, that assumes independence between the variables and applies Bayes Theorem (probability of an event, based on related conditions).

#### When do you use an AUC-ROC score? What kind of information can you gather from it?

AUC measures the area underneath the ROC curve (probability curve), providing an aggregate measure of performance across all possible classification thresholds.

#### What is cross validation?

In machine learning it is used as a performance evaluation by taking different samples (splitting the data) from the dataset, training each subsample and providing all the different scores for the model.

#### What are confounding variables?

A confounding variable is an outside variable that influences the dependent and independent variable, that is not always easy to notice (lurking variable).

#### If an important metric for our company stopped appearing in our data source, how would you investigate the causes?

## Practical Machine Learning

In this challenge, you will showcase your knowledge in feature engineering, dimensionality reduction, model selection and evaluation, hyperparameter tuning, and any other techniques of machine learning.

There isn't a correct solution to this challenge. All we would like to learn is your thinking process that demonstrates your knowledge, experience, and creativity in developing machine learning models. Therefore, in addition to developing the model and optimizing its performance, you should also elaborate your thinking process and justify your decisions thoughout the iterative problem-solving process.

The suggested time to spend on this challenge is 90-120 minutes. If you don't have time to finish all the tasks you plan to do, simply document the to-dos at the end of your response.

#### Instructions:

- Download the housing prices data set (housing_prices.csv). The data is big enough to showcase your thoughts but not so that processing power will be a problem.
- Using Python, analyze the features and determine which feature set to select for modeling.
- Train and cross validate several regression models, attempting to accurately predict the SalePrice target variable.
- Evaluate all models and show comparison of performance metrics.
- State your thoughts on model performance, which model(s) you would select, and why.

#### Deliverables Checklist:

- Python code.
- Your thinking process.
- The features selected for machine learning.
- The results (e.g., performance metrics) of your selected model(s).

#### Feature analysis and selection

In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', 100)

In [2]:
# uploading data
houses = pd.read_csv('housing_prices.csv', index_col='Id')
houses.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# NOTE: the NaNs we had before reffered to when the house didn't have a certain feature, so, the
# get_dummies dealt with it by considering it in a specific NaN column.
houses = houses.fillna(0)

In [4]:
# getting unique values length to see what object types can be coded
obj_cols = [(c, len(houses[c].unique())) for c in houses.columns if houses[c].dtype == object]
obj_cols = [c for c,v in obj_cols] # getting a list

In [5]:
# coding the objects and getting a dataset with all the features in numeric values
house_dummies = pd.get_dummies(houses[obj_cols])
houses_df = houses.drop(obj_cols, axis=1).join(house_dummies)
houses_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_0,Alley_Grvl,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,GarageType_CarPort,GarageType_Detchd,GarageFinish_0,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_0,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_0,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_0,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,Fence_0,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_0,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500,0,0,0,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500,0,0,0,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500,0,0,0,1,0,0,1,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000,0,0,0,1,0,0,1,1,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000,0,0,0,1,0,0,1,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [ ]:
houses_df.info() # all numeric now and no missing values

In [37]:
houses_df.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_0,Alley_Grvl,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,GarageType_CarPort,GarageType_Detchd,GarageFinish_0,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_0,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_0,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_0,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,Fence_0,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_0,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,57.623288,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.117123,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1868.739726,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890,0.006849,0.044521,0.010959,0.788356,0.149315,0.004110,0.995890,0.937671,0.034247,0.028082,0.331507,0.028082,0.006849,...,0.006164,0.265068,0.055479,0.241096,0.289041,0.414384,0.055479,0.002055,0.032877,0.009589,0.002055,0.897945,0.055479,0.001370,0.023973,0.006164,0.004795,0.908219,0.061644,0.020548,0.917808,0.995205,0.001370,0.001370,0.002055,0.807534,0.040411,0.036986,0.107534,0.007534,0.963014,0.001370,0.001370,0.033562,0.000685,0.029452,0.002740,0.001370,0.006164,0.003425,0.003425,0.083562,0.002055,0.867808,0.069178,0.002740,0.008219,0.013699,0.820548,0.085616
std,42.300571,34.664304,9981.264932,1.382997,1.112799,30.202904,20.645407,180.731373,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,453.697295,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883,0.082505,0.206319,0.104145,0.408614,0.356521,0.063996,0.063996,0.241835,0.181924,0.165264,0.470916,0.165264,0.082505,...,0.078298,0.441521,0.228992,0.427895

In [ ]:
# checking distribution for outliers
sns.distplot(houses_df['SalePrice'])

In [ ]:
plt.boxplot(houses_df['SalePrice'])

In [ ]:
# outliers for further use
outliers = houses_df[houses_df['SalePrice'] > 400000]

In [50]:
# getting X and y
x = houses_df.drop('SalePrice', axis=1)
y = houses_df['SalePrice']

In [8]:
# An interesting approach would be to check the features with higher correlation with the target
# for linear regression porposes
y_corr = [c for c in x.columns if abs(x[c].corr(y)) > 0.4]
len(y_corr)

26

#### Feature preparation

In [9]:
# checking collinearity
col_corr = x[y_corr].corr()[abs((x[y_corr].corr()) > 0.7) & (x[y_corr].corr() < 1)]

# getting the highly correlated features
corl = [(c, col_corr.index[col_corr[c].notnull()].tolist()) for c in col_corr.columns]
correlat = [(i,ii[0]) for i, ii in corl if len(ii)>0]
correlat = [col for pos, col in enumerate(correlat) if pos%2==0]
correlat

[('TotalBsmtSF', '1stFlrSF'),
 ('GrLivArea', 'TotRmsAbvGrd'),
 ('GarageCars', 'GarageArea')]

In [10]:
#checking which are most correlated with the target to keep, for the linear regression model
to_drop = [i if abs(x[i].corr(y)) < abs(x[ii].corr(y)) else ii for i, ii in correlat]

In [11]:
X = x[y_corr].drop(to_drop, axis=1)

In [12]:
# Another way is to do some feature reduction and scalling
features = StandardScaler().fit_transform(x[y_corr])
pca = PCA(n_components=0.99, whiten=True)

features_pca = pca.fit_transform(features)

print("Original number of features:", features.shape[1])
print("Reduced number of features:", features_pca.shape[1])
# the result is the same three I dropped, so it is good

Original number of features: 26
Reduced number of features: 23


In [13]:
# feature scaling with the dataset i will be using
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [53]:
#train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

#### Training, Testing and Evaluating Models

In [54]:
#creating a pipeline that evaluates best polynomial size
for k in range(1,5):
    poly_model = make_pipeline (StandardScaler(), PolynomialFeatures(k), LinearRegression())
    poly_model.fit(X_train, y_train)
    y_pred = poly_model.predict(X_test)
    print(k, round(mean_squared_error(y_test, y_pred),2), poly_model.score(X_test,y_test))

1 1789010539.55 0.7798054261056999
2 3.47318161561489e+30 -4.274853216339757e+20
3 2.0748386217603533e+27 -2.553747985922101e+17
4 2.7072678129358053e+19 -3332152992.55223


In [55]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('LR', round(mean_squared_error(y_test, y_pred),2), lr.score(X_test,y_test)) 

dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)
print('DTR', round(mean_squared_error(y_test, y_pred),2), dtr.score(X_test,y_test))  

knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('KNN', round(mean_squared_error(y_test, y_pred),2), knn.score(X_test,y_test))  

# The best model was linear regression
# checking cross validation for overfitting
print(cross_val_score(lr, X, y, cv=10), cross_val_score(dtr, X, y, cv=10), \
      cross_val_score(lr, X, y, cv=10))

LR 1789010539.55 0.7798054261056994
DTR 2740741401.75 0.6626647122693894
KNN 3039835329.66 0.6258517038754445
[0.84257983 0.86727381 0.87902214 0.72720839 0.82304703 0.82697203
 0.85138253 0.83920062 0.56508307 0.87112802] [0.7023411  0.74218824 0.77440546 0.60108667 0.80237084 0.79299429
 0.73823938 0.80539489 0.45290886 0.67005473] [0.84257983 0.86727381 0.87902214 0.72720839 0.82304703 0.82697203
 0.85138253 0.83920062 0.56508307 0.87112802]


In [56]:
# testing models with the full data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [57]:
for k in range(1,8):
    poly_model = make_pipeline (PolynomialFeatures(k), LinearRegression())
    poly_model.fit(X_train, y_train)
    y_pred = poly_model.predict(X_test)
    print(k, round(mean_squared_error(y_test, y_pred),2), poly_model.score(X_test,y_test))

1 600426872.99 0.91423402631873
2 7838932987.49 -0.11972623235559543


MemoryError: 

In [58]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('LR', round(mean_squared_error(y_test, y_pred),2), lr.score(X_test,y_test)) 

dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)
print('DTR', round(mean_squared_error(y_test, y_pred),2), dtr.score(X_test,y_test))  

knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('KNN', round(mean_squared_error(y_test, y_pred),2), knn.score(X_test,y_test)) 
# Aparently we have better results with the whole dataset
print(cross_val_score(lr, X, y, cv=10), cross_val_score(dtr, X, y, cv=10), \
      cross_val_score(lr, X, y, cv=10))

LR 600426872.99 0.9142340263184159
DTR 1212592635.63 0.8267912501045935
KNN 2415299039.3 0.6549946660330879
[0.84257983 0.86727381 0.87902214 0.72720839 0.82304703 0.82697203
 0.85138253 0.83920062 0.56508307 0.87112802] [0.70951684 0.74031035 0.7969757  0.47207639 0.82048354 0.75063917
 0.75295655 0.79414927 0.68477255 0.71023481] [0.84257983 0.86727381 0.87902214 0.72720839 0.82304703 0.82697203
 0.85138253 0.83920062 0.56508307 0.87112802]


In [60]:
# To check if the weird values in cross validation comes from outliers, let's use
# the dataset without them
# Dropping the outliers only from the trainning data not to influence the testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_y = X_train.join(y_train)

out_drop = X_y.drop(X_y[X_y['SalePrice'] > 400000].index)

X_train = out_drop.drop('SalePrice', axis=1)
y_train = out_drop['SalePrice']

In [ ]:
for k in range(1,8):
    poly_model = make_pipeline (PolynomialFeatures(k), LinearRegression())
    poly_model.fit(X_train, y_train)
    y_pred = poly_model.predict(X_test)
    print(k, round(mean_squared_error(y_test, y_pred),2), poly_model.score(X_test,y_test))

1 1963942934.79 0.7576370955142853
2 6384001912.19 0.21217403099001875
3 2.003592129170515e+16 -2472557.643272898
4 5.7543445498287245e+17 -71012228.21523286
5 2.7307877674367524e+18 -336996376.6068025


In [61]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('LR', round(mean_squared_error(y_test, y_pred),2), lr.score(X_test,y_test)) 

dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
y_pred = dtr.predict(X_test)
print('DTR', round(mean_squared_error(y_test, y_pred),2), dtr.score(X_test,y_test))  

knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('KNN', round(mean_squared_error(y_test, y_pred),2), knn.score(X_test,y_test)) 
# Aparently we have better results with the whole dataset
print(cross_val_score(lr, X, y, cv=10), cross_val_score(dtr, X, y, cv=10), \
      cross_val_score(lr, X, y, cv=10))

LR 1963942934.79 0.7576370955142845
DTR 3955400252.16 0.5118787432479215
KNN 3296725440.1 0.5931633558174513
[0.84257983 0.86727381 0.87902214 0.72720839 0.82304703 0.82697203
 0.85138253 0.83920062 0.56508307 0.87112802] [0.61102521 0.66096455 0.79908542 0.55428868 0.80424896 0.80238473
 0.74243556 0.79624386 0.41743386 0.699336  ] [0.84257983 0.86727381 0.87902214 0.72720839 0.82304703 0.82697203
 0.85138253 0.83920062 0.56508307 0.87112802]


#### Conclusions

I tested several models, standardizing data, selecting different columns, taking care of collinearity with different approaches, where I found out that the best model is Linear Regression,
with all the columns. Also:
* Using PCA got worse results
* Using feature scaling got worse results
* There are some weird values in cross-validation, probably due to outliers
** I tried dropping the outliers only from the training set not to influence de evaluation of the model, but I ended up getting a worse accuracy and the same weird values at cross-validation.

<br><br>
For now i would go for the Linear Regression model using the whole dataset, but for next steps I would like to figure out what is happening at the cross validation scores where we see mostly good values near the 91% accuracy, but also there is one weird value of 0.56 which needs to be taken care of.